In [7]:
!pip uninstall -y auspixdggs==1.0.0
!pip install  --upgrade  --force-reinstall git+https://github.com/CSIRO-enviro-informatics/AusPIX_DGGS.git@feat-densify-line-callable

Found existing installation: auspixdggs 1.0.0
Uninstalling auspixdggs-1.0.0:
  Successfully uninstalled auspixdggs-1.0.0
  Cloning https://github.com/CSIRO-enviro-informatics/AusPIX_DGGS.git (to revision feat-densify-line-callable) to /tmp/pip-req-build-vwgftdqt
  Running command git clone -q https://github.com/CSIRO-enviro-informatics/AusPIX_DGGS.git /tmp/pip-req-build-vwgftdqt
  Running command git checkout -b feat-densify-line-callable --track origin/feat-densify-line-callable
  Switched to a new branch 'feat-densify-line-callable'
  Branch 'feat-densify-line-callable' set up to track remote branch 'feat-densify-line-callable' from 'origin'.
  Using cached numpy-1.18.5-cp37-cp37m-manylinux1_x86_64.whl (20.1 MB)
  Using cached pyproj-2.6.1.post1-cp37-cp37m-manylinux2010_x86_64.whl (10.9 MB)
  Using cached scipy-1.4.1-cp37-cp37m-manylinux1_x86_64.whl (26.1 MB)
  Using cached Shapely-1.7.0-cp37-cp37m-manylinux1_x86_64.whl (1.8 MB)
Processing /home/jovyan/.cache/pip/wheels/a0/c5/f3/548a

In [8]:
!pip install geojson shapely pygeoj matplotlib

In [9]:
import geojson
fname = "SA_RoadsExample_lines.geojson"
geojson_data = None
with open(fname) as json_file:
    geojson_data = geojson.load(json_file)
    

In [10]:
#draw the geometry for the feature on a map
import ipyleaflet as ipy 
import ipywidgets as ipyw
from ipyleaflet import GeoJSON, Map, Marker

x_coord = -34.74
y_coord = 138.78

map1 = ipy.Map(center=[x_coord, y_coord], zoom=12)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json1 = GeoJSON(data=geojson_data, style = {'color': 'blue', 'opacity':0.8, 'weight':1.9, 'fillOpacity':0.3})
map1.add_layer(geo_json1)
map1

Map(center=[-34.74, 138.78], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

In [29]:
from shapely.geometry import shape, LineString, MultiLineString, Polygon, MultiPolygon
from geojson.utils import coords
from geojson import Feature, FeatureCollection
from auspixdggs.callablemodules.dggs_in_poly_for_geojson_callable import cells_in_poly, get_dggs_cell_bbox
from auspixdggs.callablemodules.dggs_in_line import line_to_DGGS, densify_my_line
#from auspixdggs.callablemodules.dggs_for_lines_geojson_callable import densify_my_line

def bbox(coord_list):
     box = []
     for i in (0,1):
         res = sorted(coord_list, key=lambda x:x[i])
         box.append((res[0][i],res[-1][i]))
     ret = [ box[0][0], box[1][0], box[0][1], box[1][1] ]
     return ret
    
def geojson_to_shape(g):
    return shape(g)
    
def get_cells_in_geojson(geojson, resolution):
    list_cells = []
    for fea in geojson['features']:  # for feature in attribute table
        res_cells = get_cells_in_feature(fea, resolution)
        list_cells = list(list_cells + res_cells)
    return list_cells 

def get_cells_in_feature(fea, resolution, cell_obj=True):
    geom = geojson_to_shape(fea['geometry'])
    curr_coords = list(coords(fea))
    thisbbox = bbox(curr_coords)
    cells = []
    if isinstance(geom, LineString) or isinstance(geom, MultiLineString): 
        fea['geometry']['coordinates'] = densify_my_line(fea['geometry']['coordinates'], resolution)
        curr_coords = list(coords(fea))
        print(curr_coords)        
        res_cells = line_to_DGGS(curr_coords, resolution)  # start at DGGS level 10   
        cells = res_cells
    elif isinstance(geom, Polygon) or  isinstance(geom, MultiPolygon):
        res_cells = cells_in_poly(thisbbox, fea['geometry']['coordinates'], resolution)  # start at DGGS level 10    
        print(res_cells)
        cells = [item[0] for item in res_cells]
    else: #try something anyway
        res_cells = cells_in_poly(thisbbox, fea['geometry']['coordinates'], resolution)  # start at DGGS level 10    
        print(res_cells)
        cells = [item[0] for item in res_cells]
    return cells

     

In [30]:

DGGS_RESOLUTION=10
list_cells = get_cells_in_geojson(geojson_data, DGGS_RESOLUTION)  
list_features = []
for cell in list_cells:
    bbox_coords = get_dggs_cell_bbox(cell)
    geom_obj = Polygon(bbox_coords)
    feat = Feature(geometry=geom_obj, properties={"dggs_cell_id": str(cell)}) 
    list_features.append(feat)

feature_collection = FeatureCollection(list_features)

#uncomment if you want to get a file output
#geojson_dump = geojson.dumps(feature_collection, indent=4, sort_keys=True)
#with open('poly.geojson', 'w') as json_file:
#    data = geojson.dump(feature_collection, json_file, indent=4, sort_keys=True)   

[[[138.754569, -34.712375], [138.754558, -34.712516]], [[138.754511, -34.712311], [138.754569, -34.712375]], [[138.754354, -34.712285], [138.754511, -34.712311]], [[138.754213, -34.71231], [138.754354, -34.712285]], [[138.753901, -34.712358], [138.754213, -34.71231]], [[138.75347, -34.712437], [138.753901, -34.712358]], [[138.753301, -34.712439], [138.75347, -34.712437]], [[138.752768, -34.712447], [138.753301, -34.712439]], [[138.752295, -34.712472], [138.752768, -34.712447]]]
[(138.752768, -34.712447), (138.752295, -34.712472)]
[[[138.811252, -34.717584], [138.811405, -34.717435], [138.811465, -34.717302], [138.811482, -34.717222], [138.811563, -34.717121], [138.811666, -34.717068], [138.811716, -34.716962], [138.811765, -34.716825], [138.81187, -34.716715], [138.812009, -34.716616], [138.81213, -34.716466]]]
[(138.811405, -34.717435), (138.811252, -34.717584), (138.811465, -34.717302), (138.811405, -34.717435), (138.811482, -34.717222), (138.811465, -34.717302), (138.811563, -34.717

[[[138.827869, -34.731216], [138.827755, -34.731511], [138.827612, -34.731971]]]
[(138.827755, -34.731511), (138.827869, -34.731216), (138.827612, -34.731971), (138.827755, -34.731511)]
[[[138.820406, -34.742212], [138.821019, -34.741911], [138.821234, -34.741848], [138.821344, -34.741855], [138.821511, -34.74185], [138.82163, -34.741801], [138.821752, -34.74177], [138.821985, -34.741782], [138.82215, -34.74186]]]
[(138.821019, -34.741911), (138.820406, -34.742212), (138.821234, -34.741848), (138.821019, -34.741911), (138.821344, -34.741855), (138.821234, -34.741848), (138.821511, -34.74185), (138.821344, -34.741855), (138.82163, -34.741801), (138.821511, -34.74185), (138.821752, -34.74177), (138.82163, -34.741801), (138.821985, -34.741782), (138.821752, -34.74177), (138.82215, -34.74186), (138.821985, -34.741782)]
[[[138.784435, -34.715361], [138.784653, -34.715577], [138.784867, -34.715815], [138.78516, -34.71612], [138.786582, -34.717417]]]
[(138.784653, -34.715577), (138.784435, -3

[(138.820652, -34.733579), (138.820299, -34.733618), (138.820765, -34.733563), (138.820652, -34.733579), (138.8208, -34.733488), (138.820765, -34.733563), (138.820778, -34.733418), (138.8208, -34.733488), (138.820705, -34.733359), (138.820778, -34.733418), (138.820652, -34.733309), (138.820705, -34.733359), (138.820649, -34.733222), (138.820652, -34.733309), (138.820696, -34.733153), (138.820649, -34.733222), (138.820894, -34.733048), (138.820696, -34.733153), (138.820991, -34.733029), (138.820894, -34.733048)]
[[[138.754755, -34.733969], [138.754719, -34.733744]]]
[(138.754719, -34.733744), (138.754755, -34.733969)]
[[[138.756963, -34.726572], [138.756917, -34.72645], [138.756819, -34.726374], [138.75663, -34.726312], [138.756378, -34.726263], [138.756164, -34.72625]]]
[(138.756917, -34.72645), (138.756963, -34.726572), (138.756819, -34.726374), (138.756917, -34.72645), (138.75663, -34.726312), (138.756819, -34.726374), (138.756378, -34.726263), (138.75663, -34.726312), (138.756164, -

[[[138.767679, -34.725995], [138.765188, -34.725576], [138.764687, -34.725527], [138.764291, -34.725527], [138.763867, -34.725543], [138.763326, -34.725603], [138.762278, -34.725753]]]
[(138.765188, -34.725576), (138.7656862, -34.725659799999995), (138.7661844, -34.725743599999994), (138.7666826, -34.7258274), (138.76718079999998, -34.7259112), (138.767679, -34.725995), (138.764687, -34.725527), (138.765188, -34.725576), (138.764291, -34.725527), (138.764687, -34.725527), (138.763867, -34.725543), (138.764291, -34.725527), (138.763326, -34.725603), (138.763867, -34.725543), (138.762278, -34.725753), (138.76280200000002, -34.725678), (138.763326, -34.725603)]
[[[138.825037, -34.739248], [138.824399, -34.740223], [138.824397, -34.740518]]]
[(138.824399, -34.740223), (138.82471800000002, -34.7397355), (138.825037, -34.739248), (138.824397, -34.740518), (138.824399, -34.740223)]
[[[138.812644, -34.740347], [138.812649, -34.740454], [138.812605, -34.740517], [138.812498, -34.740681], [138.8

[[[138.783311, -34.768967], [138.783164, -34.769084], [138.78307, -34.7692]]]
[(138.783164, -34.769084), (138.783311, -34.768967), (138.78307, -34.7692), (138.783164, -34.769084)]
[[[138.764299, -34.710677], [138.764311, -34.710889], [138.764333, -34.711219], [138.764364, -34.711456], [138.764426, -34.711705], [138.764475, -34.711785], [138.764564, -34.711815], [138.764708, -34.711804], [138.765405, -34.71154], [138.766112, -34.711284], [138.766354, -34.711231]]]
[(138.764311, -34.710889), (138.764299, -34.710677), (138.764333, -34.711219), (138.764311, -34.710889), (138.764364, -34.711456), (138.764333, -34.711219), (138.764426, -34.711705), (138.764364, -34.711456), (138.764475, -34.711785), (138.764426, -34.711705), (138.764564, -34.711815), (138.764475, -34.711785), (138.764708, -34.711804), (138.764564, -34.711815), (138.765405, -34.71154), (138.764708, -34.711804), (138.766112, -34.711284), (138.765405, -34.71154), (138.766354, -34.711231), (138.766112, -34.711284)]
[[[138.81615,

...

[[[138.771805, -34.739377], [138.771622, -34.73929], [138.771467, -34.739274], [138.771342, -34.739286]]]
[(138.771622, -34.73929), (138.771805, -34.739377), (138.771467, -34.739274), (138.771622, -34.73929), (138.771342, -34.739286), (138.771467, -34.739274)]
[[[138.812027, -34.710793], [138.812033, -34.710799], [138.812028, -34.710945], [138.811851, -34.711744], [138.811875, -34.711987], [138.811959, -34.71216], [138.812336, -34.712654]]]
[(138.812033, -34.710799), (138.812027, -34.710793), (138.812028, -34.710945), (138.812033, -34.710799), (138.811851, -34.711744), (138.8119395, -34.7113445), (138.812028, -34.710945), (138.811875, -34.711987), (138.811851, -34.711744), (138.811959, -34.71216), (138.811875, -34.711987), (138.812336, -34.712654), (138.811959, -34.71216)]
[[[138.812365, -34.74027], [138.81239, -34.740221], [138.81239, -34.740174]]]
[(138.81239, -34.740221), (138.812365, -34.74027), (138.81239, -34.740174), (138.81239, -34.740221)]
[[[138.799764, -34.757419], [138.7999

[[[138.770811, -34.744848], [138.770774, -34.744768], [138.770693, -34.744495]]]
[(138.770774, -34.744768), (138.770811, -34.744848), (138.770693, -34.744495), (138.770774, -34.744768)]
[[[138.754582, -34.75761], [138.754542, -34.757705], [138.754502, -34.757797], [138.754463, -34.757885], [138.754423, -34.757975], [138.754385, -34.758063], [138.754348, -34.758157], [138.754312, -34.758252], [138.754273, -34.758348], [138.754232, -34.758445], [138.75419, -34.758542], [138.754148, -34.758638], [138.754108, -34.758735], [138.754068, -34.758828], [138.754025, -34.75892], [138.753983, -34.75901], [138.753943, -34.7591], [138.753903, -34.75919], [138.753865, -34.759273], [138.753832, -34.759348], [138.753803, -34.759422], [138.753775, -34.759495], [138.753748, -34.75957], [138.753718, -34.759645], [138.753685, -34.759718], [138.753642, -34.75979], [138.753592, -34.759857], [138.753545, -34.75992], [138.753507, -34.759985], [138.753478, -34.760055], [138.75346, -34.76013], [138.753442, -34.7

[[[138.811178, -34.72932], [138.811293, -34.729306], [138.811419, -34.729223], [138.81178, -34.728949], [138.811836, -34.728841], [138.811838, -34.728612]]]
[(138.811293, -34.729306), (138.811178, -34.72932), (138.811419, -34.729223), (138.811293, -34.729306), (138.81178, -34.728949), (138.811419, -34.729223), (138.811836, -34.728841), (138.81178, -34.728949), (138.811838, -34.728612), (138.811836, -34.728841)]
[[[138.762563, -34.738679], [138.762558, -34.738795], [138.762554, -34.739102], [138.762553, -34.739242], [138.762556, -34.73976], [138.762536, -34.739891], [138.762509, -34.74003], [138.762421, -34.740317], [138.762305, -34.740582], [138.762262, -34.740679], [138.762106, -34.741151], [138.762, -34.741485], [138.761891, -34.741797], [138.761767, -34.742146], [138.761685, -34.742263], [138.761269, -34.742508], [138.760778, -34.742806], [138.76062, -34.742862], [138.760487, -34.742858], [138.760129, -34.742776], [138.759785, -34.742689]]]
[(138.762558, -34.738795), (138.762563, -3

[[[138.783929, -34.776186], [138.783906, -34.776102], [138.783918, -34.776033], [138.783956, -34.775958], [138.784045, -34.775831], [138.784231, -34.775565], [138.78445, -34.775292], [138.784524, -34.775176], [138.7846, -34.775021], [138.784675, -34.774839], [138.784762, -34.774628], [138.784868, -34.774425], [138.784981, -34.774207], [138.785134, -34.773958], [138.785184, -34.773859], [138.785195, -34.773837], [138.785215, -34.773754], [138.785234, -34.773654], [138.785231, -34.773276], [138.785193, -34.77278], [138.785193, -34.772363], [138.785214, -34.772161], [138.785205, -34.771965], [138.785158, -34.771524], [138.785063, -34.771097], [138.784993, -34.770957]]]
[(138.783906, -34.776102), (138.783929, -34.776186), (138.783918, -34.776033), (138.783906, -34.776102), (138.783956, -34.775958), (138.783918, -34.776033), (138.784045, -34.775831), (138.783956, -34.775958), (138.784231, -34.775565), (138.784045, -34.775831), (138.78445, -34.775292), (138.784231, -34.775565), (138.784524, 

[[[138.801991, -34.748353], [138.802014, -34.748267], [138.802165, -34.748141], [138.802448, -34.748061], [138.802783, -34.747979], [138.803082, -34.747863], [138.803227, -34.747801], [138.80338, -34.747725], [138.803386, -34.747653], [138.803356, -34.747575], [138.803221, -34.747443], [138.803165, -34.747373], [138.803136, -34.747295], [138.803211, -34.747214], [138.80326, -34.747141], [138.803274, -34.747062], [138.80324, -34.746887], [138.803357, -34.746776], [138.803585, -34.746619], [138.803686, -34.746544], [138.803851, -34.746554], [138.804007, -34.746542], [138.804226, -34.746579], [138.804358, -34.746618], [138.804435, -34.746601], [138.80446, -34.746507], [138.804402, -34.746366], [138.804308, -34.746196], [138.804289, -34.745968], [138.804295, -34.745752], [138.804293, -34.745638], [138.804425, -34.745541], [138.80451, -34.74551], [138.804791, -34.745516], [138.804967, -34.745561], [138.805143, -34.745613], [138.805405, -34.745663], [138.805603, -34.745815], [138.805808, -34

In [31]:
map2 = ipy.Map(center=[x_coord, y_coord], zoom=12)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json2 = GeoJSON(data=feature_collection, 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'red', 
                          'fillOpacity': 0.3})
map2.add_layer(geo_json2)
map2.add_layer(geo_json1)
map2

Map(center=[-34.74, 138.78], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

In [7]:
import requests
import json
#iterate over features in feature collection and query Loc-I API for features
def find_features_at_dggs_cell(dggs_cell_id):
    payload = {
        'dggs_cell' : dggs_cell_id
    }
    r = requests.get('https://api.loci.cat/api/v1/location/find-at-DGGS-cell', params=payload)
    json_res = r.json()
    return json_res


In [8]:
target_feature_type = 'asgs16_sa1'

set_of_sa1_features = set([])
for feature in feature_collection['features']:
    cell_id = feature['properties']['dggs_cell_id']
    print("Processing {}".format(cell_id))
    res = find_features_at_dggs_cell(cell_id)
    if 'locations' in res:
        for loc in res['locations']:
            if 'dataType' in loc and loc['dataType'] == target_feature_type:
                set_of_sa1_features.add(loc['uri'])
                print("Adding {}".format(loc['uri']))

    


Processing R7751215231
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103304
Processing R7751215230
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103304
Processing R7751224073
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751224070
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751218457
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751218458
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751218455
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751218533
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751218530
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751218506
Adding http://

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226355
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226358
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226382
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226385
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226388
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226622
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226621
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751215283
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103309
Processing R7751215280
Adding http://linked.data.gov.au/data

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751223640
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751223616
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751224661
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751224637
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751227138
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227444
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227445
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227448
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227456
Adding http://linked.data.gov.au/data

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103303
Processing R7751215750
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103303
Processing R7751224654
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309
Processing R7751224651
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309
Processing R7751224652
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309
Processing R7751224470
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309
Processing R7751224471
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309
Processing R7751223517
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223514
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751224641
Adding http://linked.data.gov.au/data

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227345
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227344
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227347
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227371
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227374
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751218523
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751218524
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751218521
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751218520
Adding http://linked.data.gov.au/data

...

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751226240
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751226241
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751218233
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751218236
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751218237
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751227177
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227178
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227186
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751224175
Adding http://linked.data.gov.au/data

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309
Processing R7751227130
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751224673
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751224665
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751224664
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751224661
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751226262
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751226238
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751226246
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223331
Adding http://linked.data.gov.au/data

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223855
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223854
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223853
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223845
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223830
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223752
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751224384
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309
Processing R7751226266
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751226501
Adding http://linked.data.gov.au/data

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103307
Processing R7751226443
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226444
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226445
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226453
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100312
Processing R7751224720
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309
Processing R7751224712
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309
Processing R7751224478
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309
Processing R7751224650
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309
Processing R7751224627
Adding http://linked.data.gov.au/data

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751224661
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305


In [9]:
print(json.dumps(list(set_of_sa1_features), indent=3))

[
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103307",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100312",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103304",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100322",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103303",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103306",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103309",
  

In [10]:
def get_geom(loci_uri):
    payload = {
        "uri": loci_uri
    }
    url = "https://api.loci.cat/api/v1/location/geometry"
    r = requests.get(url, params=payload)
    res = r.json()
    #get the first geom result
    geojson_data = []
    if len( res['geometry']) > 0:
        geojson_data = res['geometry'][0]
    return geojson_data




In [11]:
map3 = ipy.Map(center=[x_coord, y_coord], zoom=12)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))


In [12]:
for feature_uri in list(set_of_sa1_features):
    curr_geojson_layer = GeoJSON(data=get_geom(feature_uri), 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'yellow', 
                          'fillOpacity': 0.3})
    map3.add_layer(curr_geojson_layer)


In [13]:
map3.add_layer(geo_json1)



In [14]:
map3

Map(center=[-34.74, 138.78], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…